In [0]:
%restart_python

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, year, month, dayofmonth, hour, to_timestamp, when, lit
from pyspark.sql.types import ArrayType, DoubleType,StringType
import matplotlib.pyplot as plt
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
import seaborn as sns
import pandas as pd

from utils._categorical_summary import cat_summary
from utils._check_dataframe import check_dataframe
from utils._descriptive_statistics import desc_stats
from utils._num_summary import num_summary
from utils._categorical_with_numerical import calculate_aggregation, cat_with_numerical

In [0]:
catalog_name = 'samples'
schema_name = 'nyctaxi'
table_name = 'trips'

full_table_path = f'{catalog_name}.{schema_name}.{table_name}'

df = spark.table(full_table_path)

In [0]:
check_dataframe(df)

- In this output, we can easily see the general outline of the data set. Before analyzing the data set we have, we need to perform some simple operations on it so that the insights we gain from the data are accurate.

- In this context, we will first convert the data types of some columns in the dataset from numerical to string format (see pickup_zipcode, dropoff_zipcode). After that, we have two important variables that can provide us with very important information. These are pickup_datetime and dropoff_datetime. With these two variables, we will obtain the duration of the trip in minutes. By establishing an interaction between the trip_duration_minute variable we created with trip_distance, we will obtain the average hourly speed value. 

- Using these new variables, we will perform outlier analysis on the dataset, removing illogical cases from the dataset or suppressing them appropriately.

In [0]:
df = df.withColumn('pickup_zip', F.col('pickup_zip').cast('string'))
df = df.withColumn('dropoff_zip', F.col('dropoff_zip').cast('string') )

In [0]:
df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
        .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime") \
        .withColumn("trip_duration_minutes", (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long")) / 60 ) \
        .withColumn('trip_duration_minutes', F.round(F.col('trip_duration_minutes'), 2))

In [0]:
df = df.withColumn(
    "avg_speed_mph",
    F.when(
        F.col("trip_duration_minutes") > 0,  
        F.col("trip_distance") / (F.col("trip_duration_minutes") / 60) 
    ).otherwise(
        F.lit(None).cast("double")
    )
).withColumn(
    'avg_speed_mph',
    F.round(F.col('avg_speed_mph'), 3)
)

In [0]:
df = df.withColumn(
    "Trip_Type", 
    F.when(
        
        F.col("pickup_zip") == F.col("dropoff_zip"), 
        F.lit("In-Zone") 
    ).otherwise(
        F.lit("Out-of-Zone") 
    )
)

In [0]:
df = df.withColumn(
        'zip_lower',
        F.when(
            F.col('pickup_zip').cast('string') <= F.col('dropoff_zip').cast('string'), 
            F.col('pickup_zip')
        ).otherwise(F.col('dropoff_zip'))
    )


df = df.withColumn(
        'zip_greater',
        F.when(
            F.col('pickup_zip').cast('string') > F.col('dropoff_zip').cast('string'), 
            F.col('pickup_zip')
        ).otherwise(F.col('dropoff_zip'))
    )

df = df.withColumn(
        'Combined_Zip',
        F.concat(
            F.col('zip_lower').cast('string'), 
            F.lit(' - '), 
            F.col('zip_greater').cast('string')
        ).cast(StringType())
    )
df = df.drop('zip_lower', 'zip_greater')

In [0]:
check_dataframe(df)

In [0]:
numerical_cols = ['trip_distance', 'fare_amount', 'trip_duration_minutes', 'avg_speed_mph']

for col_name in numerical_cols:
    df = df.withColumn(col_name, F.col(col_name).cast(DoubleType()))

desc_stats(df, numerical_cols)

- When we look at the distribution of numeric variables here, we can see obvious errors in our data.

- For example, in the variable named fare_amount, the minimum value is -8. A negative value in an amount variable clearly indicates incorrect data entry.

- Another error occurs between the trip_distance and trip_duration_minutes variables. The trip_distance variable has a skewed distribution with a mean of 2.85, a standard deviation of 3.44, and a mean of 30.60. However, when we look at the trip_duration_minutes variable, we see a variable with a mean of 15.12 and a standard deviation of 52.84, ranging from 0 to 1438.85. The fact that a 1438-minute journey is 30.60 miles seems far from reality. There are likely outliers or incorrect data entries.              

In [0]:
num_summary(df, numerical_cols=["trip_distance"], plot=True)

- In the Trip_Distance variable, the data ranges from 0 to 30.6. The mean is 2.85 and the standard deviation is 3.43.

- We can clearly say that this distribution contains outliers. We need to set thresholds to minimize the loss of rows and remove values ​​above and below these thresholds from the dataset.

- To achieve this, I tested various threshold values ​​and found that the threshold values ​​that resulted in the least information loss were [0.035, 0.965]. I begin the data cleaning process by removing values ​​below and above these values ​​from the dataset.

In [0]:
quantiles_distance = df.approxQuantile("trip_distance", [0.035, 0.965], 0.01)

lower_bound_distance = quantiles_distance[0]  
upper_bound_distance = quantiles_distance[1]

df_filtered_distance = df.filter(
    (F.col("trip_distance") > lower_bound_distance) & 
    (F.col("trip_distance") < upper_bound_distance)
)
num_summary(df_filtered_distance, numerical_cols=["trip_distance"], plot=True)

- When we remove rows from the dataset based on threshold values, we can see that our trip_distance values ​​are distributed between [0.41, 10.78], with a mean of 2.35 and a standard of 2.03.

- We can see that our total row count has decreased from 21,932 to 20,278. 
  * (Of course, this type of cleaning might be considered crude by the general approach, but we should keep in mind that this project is an introductory project to PySpark and DataBricks.)

In [0]:
num_summary(df_filtered_distance, numerical_cols=["trip_duration_minutes"], plot=True)
desc_stats(df_filtered_distance, numerical_cols=["trip_duration_minutes"])

- Our next numerical variable is trip_duration_minutes. This variable isn't a variable found in the dataset; we calculated the duration of the taxi trip by taking the difference between the pickup_datetime and drop-off_datetime variables.

- When I examine the data distribution, I see that it's distributed between [0.03, 1438.85], with a mean of 14.21 and a standard deviation of 51.27. This distribution, like the trip_distance variable, is skewed. Therefore, we'll find appropriate thresholds and perform row deletion to minimize information loss whenever possible.

- We can also perform suppression, but since we're not building an ML model, I'm currently cleaning the data by removing outliers.

In [0]:
quantiles_mins = df.approxQuantile("trip_duration_minutes", [0.02, 0.98], 0.01)

lower_bound_mins = quantiles_mins[0]  
upper_bound_mins = quantiles_mins[1]

df_filtered_mins = df_filtered_distance.filter(
    (F.col("trip_duration_minutes") > lower_bound_mins) & 
    (F.col("trip_duration_minutes") < upper_bound_mins)
)
num_summary(df_filtered_mins, numerical_cols=["trip_duration_minutes"], plot=True)
desc_stats(df_filtered_mins, numerical_cols=["trip_duration_minutes"])

- I set my threshold values ​​to [0.2, 0.98], and our total row count dropped to 19,841 rows. The number of observations in the dataset will continue to decrease, but decisions based on inferences derived from inaccurate data can incur significant costs. While obtaining clean data can be quite difficult and costly in real-world situations, working with inaccurate data and making business decisions can be even more costly.

In [0]:
num_summary(df_filtered_mins, ['avg_speed_mph'], plot=True)
desc_stats(df_filtered_mins, numerical_cols=["avg_speed_mph"])

- After cleaning up the Trip_Distance and Trip_Duration_Minutes variables, the avg_speed_mph variable actually looks quite clean. Of course, a speed of 75.97 mph may be relatively high or low, but at this point, we can proceed as if there were no outliers in this variable.

In [0]:
num_summary(df_filtered_mins, ["fare_amount"], True)
desc_stats(df_filtered_mins, numerical_cols=["fare_amount"])

- I see some errors in the fare_amount variable. Looking at the overall distribution, there are values ​​less than 0 and values ​​like $65, which could be considered outliers but are considered very high compared to the overall distribution. I will remove these from the dataset as well.

In [0]:
quantiles_famount = df_filtered_mins.approxQuantile("fare_amount", [0.02,0.98], 0.01)
lower_bound_famount = quantiles_famount[0]  
upper_bound_famount = quantiles_famount[1]

df_filtered_famount = df_filtered_mins.filter(
    (F.col("fare_amount") > lower_bound_mins) & 
    (F.col("fare_amount") < upper_bound_mins)
)
num_summary(df_filtered_famount, numerical_cols=["fare_amount"], plot=True)
desc_stats(df_filtered_famount, numerical_cols=["fare_amount"])

- I found that the 2% and 98% percentile values ​​for Fare_Amount were suitable thresholds for us. I removed any values ​​above or below these thresholds from the dataset, and now the dataset is numerically clean.

- Contextually, we will determine whether the data contains logical errors when analyzing the categorical variables. However, the fare_amount variable currently has a distribution between $3.5 and $38, with a mean of $10.85 and a standard deviation of $5.92.

In [0]:
df = df_filtered_famount

In [0]:
df_ts = df.withColumn(
    "pickup_datetime", 
    to_timestamp(col("pickup_datetime"), "yyyy-MM-dd HH:mm:ss")
).withColumn(
    "dropoff_datetime", 
    to_timestamp(col("dropoff_datetime"), "yyyy-MM-dd HH:mm:ss")
)

In [0]:
df_final = df_ts.withColumn("pickup_month", month(col("pickup_datetime"))) \
                .withColumn("pickup_day", dayofmonth(col("pickup_datetime"))) \
                .withColumn("pickup_hour_int", hour(col("pickup_datetime")))

time_slot_col = F.lit(None).cast("string")

In [0]:
for h in range(24):
        start_hour = h
        end_hour = (h + 1) % 24 
        
        if start_hour == 0:
            start_str = "12:00 AM"
        elif start_hour < 12:
            start_str = f"{start_hour:02d}:00 AM"
        elif start_hour == 12:
            start_str = "12:00 PM"
        else:
            start_str = f"{(start_hour - 12):02d}:00 PM"
            
        if end_hour == 0:
            end_str = "12:00 AM"
        elif end_hour < 12:
            end_str = f"{end_hour:02d}:00 AM"
        elif end_hour == 12:
            end_str = "12:00 PM"
        else: 
            end_str = f"{(end_hour - 12):02d}:00 PM"
        
        
        slot_label = f"{start_str} - {end_str}"
        
        time_slot_col = F.when(
            F.col("pickup_hour_int") == start_hour, 
            slot_label
        ).otherwise(time_slot_col)

df_final = df_final.withColumn("pickup_time_category", time_slot_col)

df_final = df_final.withColumn(
            "pickup_am_pm",
            when(col("pickup_hour_int") < 12, lit("AM"))
            .otherwise(lit("PM"))
)
df_final = df_final.drop("pickup_hour_int")

In [0]:
cat_summary(df_final, ['pickup_month','pickup_day','pickup_time_category', 'pickup_am_pm', 'Trip_Type', 'Combined_Zip'], plot= True)

- When I examine the dataset specifically for categorical variables, I see that we are working on an analysis covering only January and February of 2016. The number of trips made in January and February is very similar (50.64% in February, 49.36% in January). When I examine the frequency distribution of the trips made in our study by day, I see that the highest number of trips occurred on the 13th day. However, we don't know which month this value occurred on the 13th, which we will examine later.

- I see that the number of trips made in the afternoon (PM) is significantly higher than the number of trips made before noon (AM). When I plot the frequency breakdown by time of day for the Pickup_Date variable, we clearly see that more trips occur in the second half of the day.

- Above, I created a Trip_Type variable using the pickup_zipcode and dropp_off_zipcode variables. This variable stores information about the In-Zone if the zip code of a taxi driver's pickup location matches the zip code of the drop-off location; otherwise, the Out-Of-Zone information is stored. When I plot the data frequency using this variable, I see that Out-Of-Zone values ​​dominate.

- When I plot the frequency using the Combined_Zip variable, the final categorical variable in my dataset, which combines the Pickup_Zipcode and Drop_Off_Zipcode values, we see that the most trips are made from zip codes such as 10028, 10021, 10023, 10003, and 10065.

In [0]:
cat_summary(df_final, ['pickup_day','pickup_time_category', 'pickup_am_pm', 'Trip_Type', 'Combined_Zip'], plot= True, hue= 'pickup_month')

In [0]:
cat_with_numerical(df_final, 'pickup_month','trip_distance','median',top_n = 10 )


- When I examine the two graphs, we see that the distribution of the trip_distance variable for the two months in the data set is very similar. Furthermore, when we examine the frequency distribution, we see that the number of trips between the two months is also very similar. When we examine the median values, we see that more trips were made in February than in January.

In [0]:
cat_with_numerical(df_final, 'pickup_month','trip_duration_minutes','median',top_n = 10 )

In [0]:
cat_with_numerical(df_final, 'pickup_month','trip_duration_minutes','sum',top_n = 10 )


- When we examine the Trip_Duration_Minutes variable and the monthly distribution graphs, we see that the distributions for the two months are almost identical. The median values ​​are also equal. However, when we examine the total values, we see that the total travel time in February is higher than in January.


In [0]:
cat_with_numerical(df_final, 'pickup_month','fare_amount','median',top_n = 10 )

In [0]:
cat_with_numerical(df_final, 'pickup_month','fare_amount','sum',top_n = 10 )


- When we examine the fare amount variable and the monthly distribution graphs, we see that the distributions for the two months are almost identical. The median values are also equal. However, when we examine the total values, we see that the total travel time in February is higher than in January.

In [0]:
cat_with_numerical(df_final, 'pickup_month','avg_speed_mph','median',top_n = 10 )


- In these two graphs comparing monthly average speeds, we can see that the average speed for trips in January was higher than in the previous month.

- We observed a higher number of taxi trips in February. Due to the increased traffic congestion in February, we can assume that the average speed for vehicles during trips is naturally lower.

In [0]:
cat_with_numerical(df_final, 'Trip_Type','trip_distance','median',top_n = 10 )


- When I examine the scatter plots of the trip_type breakdown for the trip distance variable, I can clearly see that the out-of-zone values ​​have a wider distribution (left), while the in-zone values ​​have a more compact distribution. When we compare their medians, we see that the out-of-zone values ​​have a higher median.

- However, it's important to note that there's a class imbalance for this variable. Our out-of-zone values ​​are more dominant than the other values ​​in the dataset. We need to keep this in mind when evaluating the other numerical breakdowns we'll perform for Trip_Type.

In [0]:
cat_with_numerical(df_final, 'Trip_Type','trip_duration_minutes','median',top_n = 10 )

In [0]:
cat_with_numerical(df_final, 'Trip_Type','avg_speed_mph','median',top_n=10)

In [0]:
cat_with_numerical(df_final, 'Trip_Type','fare_amount','sum',top_n=10)

In [0]:
cat_with_numerical(df_final, 'Combined_Zip','fare_amount','sum',top_n=10)

In [0]:
cat_with_numerical(df_final, 'Combined_Zip','fare_amount','median',top_n=10)

- When we examine the Combined_Zip and Fare_Amount variables from the perspective of both total and median values, we encounter an interesting result. While the 10021-10028 route, which has the highest frequency in the data set, has the highest total fare_amount compared to other travel routes, its median value is in the lower middle range.

- Meanwhile, the 10021-10065 route, which ranks fifth in the data set in terms of highest frequency, has the lowest total fare_amount and median values.

In [0]:
cat_with_numerical(df_final, 'pickup_time_category','trip_distance','median',top_n=10)

In [0]:
cat_with_numerical(df_final, 'pickup_time_category','trip_distance','sum',top_n=10)

- When we examine the pickup_time_category variable for various numerical variables, we see that, on average, the longest trips generally occur between the hours of 10:00 PM - 11:00 PM, 9:00 PM - 10:00 PM, and 8:00 PM - 9:00 PM.

- However, when I examine the total trip_distance values, I see that the longest trip time is between 6:00 PM - 7:00 PM, with a distance of 2915.52 miles. The average for this time period is 1.58.

In [0]:
cat_with_numerical(df_final, 'pickup_time_category','trip_duration_minutes','median',top_n=10)

In [0]:
cat_with_numerical(df_final, 'pickup_time_category','trip_duration_minutes','sum',top_n=10)

- When we compare our trip_distance_minutes variable with time zones, we obtain information that confirms the previous graph. The time zone with the most time is between 06:00 PM and 07:00 PM.